# Geographical Data Preparation

In this notebook we clean and prepare the dataset, which contains information about the bike sharing stations within the divvy bike rental network.

## 1 Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

%matplotlib inline

Load Bike Dataset

In [2]:
df_chicago = pd.read_csv('../dataset/chicago_2018_clean.csv') 
df_chicago.head()

start_time             end_time  start_station_id  end_station_id  \
0  2018-04-01 00:04:44  2018-04-01 00:13:03                22             171   
1  2018-04-01 00:06:42  2018-04-01 00:27:07               157             190   
2  2018-04-01 00:07:19  2018-04-01 00:23:19               106             106   
3  2018-04-01 00:07:33  2018-04-01 00:14:47               241             171   
4  2018-04-01 00:10:23  2018-04-01 00:22:12               228             219   

               start_station_name                end_station_name  bike_id  \
0              May St & Taylor St           May St & Cullerton St     3819   
1  Lake Shore Dr & Wellington Ave  Southport Ave & Wrightwood Ave     5000   
2           State St & Pearson St           State St & Pearson St     5165   
3             Morgan St & Polk St           May St & Cullerton St     3851   
4         Damen Ave & Melrose Ave         Damen Ave & Cortland St     5065   

    user_type duration_per_trip  trip_time_in_hours  
0  Subscriber   0 days 00:08:19            0.138611  
1  Subscriber   0 days 00:20:25            0.340278  
2    Customer   0 days 00:16:00            0.266667  
3  Subscriber   0 days 00:07:14            0.120556  
4  Subscriber   0 days 00:11:49            0.196944

## 2 Load Station Dataset

We retrieved the geographical data for the bike stations from the [Chicago Data Portal](https://data.cityofchicago.org/). They provide a dataset which contains a list of all the stations where one can pick up and return bicycles from the Divvy bicycle sharing system, which is the same system the data for the bike rides is based on. The particular dataset can be accessed [here](https://data.cityofchicago.org/Transportation/Divvy-Bicycle-Stations/bbyy-e7gq/data). According to the website it was last updated in 2022, meaning it contains newer data than the data our biked dataset is based on. However, as long as only new stations were added this should not pose a problem. We also later consider the case of stations being altered.

In [3]:
df_chicago_stations = pd.read_csv('../dataset/Divvy_Bicycle_Stations.csv')
df_chicago_stations.head()

ID                             Station Name  Total Docks  \
0  1594046383808271024                   Troy St & Jackson Blvd            9   
1                  641      Central Park Ave & Bloomingdale Ave           11   
2  1683527931525155814      Public Rack - Cornell Ave & 87th Pl            1   
3                  367                     Racine Ave & 35th St           15   
4  1673852313397164648  Public Rack - Francisco Ave & Touhy Ave            2   

   Docks in Service      Status   Latitude  Longitude  \
0                 9  In Service  41.877505 -87.704850   
1                11  In Service  41.914166 -87.716755   
2                 1  In Service  41.736881 -87.583146   
3                15  In Service  41.830689 -87.656211   
4                 2  In Service  42.011865 -87.701317   

                                  Location  
0                   (41.877505, -87.70485)  
1                  (41.914166, -87.716755)  
2              (41.73688127, -87.58314552)  
3  (41.83068856472101, -87.65621066093445)  
4                  (42.011865, -87.701317)

## 3 NaN Values

In [4]:
df_chicago_stations.isnull().sum()

ID                  0
Station Name        0
Total Docks         0
Docks in Service    0
Status              0
Latitude            0
Longitude           0
Location            0
dtype: int64

The dataset has no missing values.

## 4 Duplicates

In [5]:
df_chicago_stations[df_chicago_stations.duplicated(keep=False)]

Empty DataFrame
Columns: [ID, Station Name, Total Docks, Docks in Service, Status, Latitude, Longitude, Location]
Index: []

The data contains no duplicate rows.

## 5 Completeness

In [6]:
relevant_station_ids = np.union1d(
    df_chicago['start_station_id'].unique(), df_chicago['end_station_id'].unique()
)
relevant_station_ids

array([  2,   3,   4,   5,   6,   7,   9,  11,  12,  13,  14,  15,  16,
        17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,
        30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
        43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
        56,  57,  58,  59,  60,  61,  62,  66,  67,  68,  69,  71,  72,
        73,  74,  75,  76,  77,  80,  81,  84,  85,  86,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 152, 153, 154, 156, 157, 158, 159,
       160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172,
       173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185,
       186, 188, 190, 191, 192, 193, 194, 195, 196, 197, 198, 19

These are the ids of all unique stations in the bike dataset. Firstly, we check whether or not the station dataset covers all of these stations.

In [7]:
available_stations_ids = np.intersect1d(df_chicago_stations['ID'].unique(), 
                                    relevant_station_ids)

In [8]:
missing_stations_ids = np.setdiff1d(relevant_station_ids, available_stations_ids)
print("Missing station ids:",missing_stations_ids)
print("We do not have data for", len(missing_stations_ids), "stations.")

Missing station ids: [360 361 363 397 459 512 606 607 608 609 610 611 612 613 614 615 616 617
 618 651]
We do not have data for 20 stations.


In [9]:
missing_stations = {}

for station_id in missing_stations_ids:
    station_name = np.nan
    if not df_chicago[df_chicago['start_station_id'] == station_id].empty:
        station_name = df_chicago[df_chicago['start_station_id'] == station_id]['start_station_name'].iloc[0]
    elif not df_chicago[df_chicago['end_station_id'] == station_id].empty:
        station_name = df_chicago[df_chicago['end_station_id'] == station_id]['end_station_name'].iloc[0]
    missing_stations[station_id] = station_name
missing_stations

{360: 'DIVVY Map Frame B/C Station',
 361: 'DIVVY CASSETTE REPAIR MOBILE STATION',
 363: 'TS ~ DIVVY PARTS TESTING',
 397: 'Saginaw Ave & Exchange Ave',
 459: 'Lakefront Trail & Bryn Mawr Ave',
 512: 'BBB ~ Divvy Parts Testing',
 606: 'Forest Ave & Chicago Ave',
 607: 'Cuyler Ave & Augusta St',
 608: 'Humphrey Ave & Ontario St',
 609: 'Forest Ave & Lake St',
 610: 'Marion St & South Blvd',
 611: 'Oak Park Ave & South Blvd',
 612: 'Ridgeland Ave & Lake St',
 613: 'Wisconsin Ave & Madison St (Temp)',
 614: 'East Ave & Madison St',
 615: 'Lombard Ave & Madison St',
 616: 'Oak Park Ave & Harrison St',
 617: 'East Ave & Garfield St',
 618: 'Lombard Ave & Garfield St',
 651: 'Michigan Ave & 71st St'}

Above are the stations which are not included in the stations dataset. In total these are 20 stations. Looking at some of the names one can see that some of these stations are operational Divvy stations which are e.g used for repairing or testing bikes. It is possible that divvy removed or even renamed some of these stations after 2018 thus their absence in the station dataset. However, some of these are legitimate streets. Their absence could mean that they either lost their status as divvy bicycles stations or their id changed for some reason.

E.g. the station 'Michigan Ave & 71st St' can be found in the station dataset, however its id changed from 651 to 674. 

In [10]:
df_chicago_stations[
    df_chicago_stations['Station Name'].str.contains('Michigan Ave & 71st St')
]

ID            Station Name  Total Docks  Docks in Service      Status  \
21  674  Michigan Ave & 71st St            7                 7  In Service   

     Latitude  Longitude                 Location  
21  41.765286 -87.621748  (41.765286, -87.621748)

The same goes for the stations 'Lakefront Trail & Bryn Mawr Ave' and 'Michigan Ave & 71st St'.

In [11]:
df_chicago_stations[
    df_chicago_stations['Station Name'].str.contains('Lakefront Trail & Bryn Mawr Ave')
]

ID                     Station Name  Total Docks  Docks in Service  \
676  760  Lakefront Trail & Bryn Mawr Ave           19                19   

         Status   Latitude  Longitude                                 Location  
676  In Service  41.984044 -87.652281  (41.98404411519213, -87.65228122472763)

In [12]:
df_chicago_stations[
    df_chicago_stations['Station Name'].str.contains('Michigan Ave & 71st St')
]

ID            Station Name  Total Docks  Docks in Service      Status  \
21  674  Michigan Ave & 71st St            7                 7  In Service   

     Latitude  Longitude                 Location  
21  41.765286 -87.621748  (41.765286, -87.621748)

Even for these stations however there might be a justified reason for the change of id. For that reason we will treat these stations the same way as all the other stations which we cannot map to the station dataset.

## 6 Synthesise Station Dataset

### 6.1 Rename columns and index by id

In [13]:
df_chicago_stations_indexed_by_id = df_chicago_stations.set_index('ID')
df_chicago_stations_indexed_by_id = df_chicago_stations_indexed_by_id.drop(
    ['Total Docks', 'Docks in Service', 'Status'], axis = 1
)
df_chicago_stations_indexed_by_id = df_chicago_stations_indexed_by_id.rename(
    columns = {'Latitude': 'x', 'Longitude': 'y', 'Station Name': 'station_name', 'Location': 'position'}
)
df_chicago_stations_indexed_by_id.head()

station_name          x  \
ID                                                                        
1594046383808271024                   Troy St & Jackson Blvd  41.877505   
641                      Central Park Ave & Bloomingdale Ave  41.914166   
1683527931525155814      Public Rack - Cornell Ave & 87th Pl  41.736881   
367                                     Racine Ave & 35th St  41.830689   
1673852313397164648  Public Rack - Francisco Ave & Touhy Ave  42.011865   

                             y                                 position  
ID                                                                       
1594046383808271024 -87.704850                   (41.877505, -87.70485)  
641                 -87.716755                  (41.914166, -87.716755)  
1683527931525155814 -87.583146              (41.73688127, -87.58314552)  
367                 -87.656211  (41.83068856472101, -87.65621066093445)  
1673852313397164648 -87.701317                  (42.011865, -87.701317)

### 6.2 Add available data

First we query the data for all stations available in the dataset and add them to a new dataframe.

In [14]:
df_stations = df_chicago_stations_indexed_by_id.loc[available_stations_ids]
df_stations.head()

station_name          x          y  \
ID                                                 
2      Buckingham Fountain  41.876423 -87.620339   
3           Shedd Aquarium  41.867226 -87.615355   
4           Burnham Harbor  41.857412 -87.613792   
5   State St & Harrison St  41.874053 -87.627716   
6           Dusable Harbor  41.886976 -87.612813   

                                   position  
ID                                           
2                   (41.876423, -87.620339)  
3          (41.86722595682, -87.6153553902)  
4   (41.85741178707404, -87.61379152536392)  
5                   (41.874053, -87.627716)  
6                   (41.886976, -87.612813)

### 6.3 Fill missing stations with NaN

Then, we add a row for every missing stations, filling every column except the id and name column with NaN values. Since we do not have the data for these stations this is our only option. It is not feasible to e.g. estimate the position of a station, since this is not a feature that we can derive from any other property of the station or other realted entries in the dataset. When using this dataset in other tasks we make sure to consider the presence of these NaN values.

In [15]:
df_missing_stations = pd.DataFrame(
    data=missing_stations.values(), index=missing_stations.keys(), columns=['station_name']
)
df_missing_stations.index.name = 'ID'

In [16]:
df_stations = pd.concat([df_stations, df_missing_stations])
df_stations

station_name          x          y  \
ID                                                      
2           Buckingham Fountain  41.876423 -87.620339   
3                Shedd Aquarium  41.867226 -87.615355   
4                Burnham Harbor  41.857412 -87.613792   
5        State St & Harrison St  41.874053 -87.627716   
6                Dusable Harbor  41.886976 -87.612813   
..                          ...        ...        ...   
615    Lombard Ave & Madison St        NaN        NaN   
616  Oak Park Ave & Harrison St        NaN        NaN   
617      East Ave & Garfield St        NaN        NaN   
618   Lombard Ave & Garfield St        NaN        NaN   
651      Michigan Ave & 71st St        NaN        NaN   

                                    position  
ID                                            
2                    (41.876423, -87.620339)  
3           (41.86722595682, -87.6153553902)  
4    (41.85741178707404, -87.61379152536392)  
5                    (41.874053, -87.627716)  
6                    (41.886976, -87.612813)  
..                                       ...  
615                                      NaN  
616                                      NaN  
617                                      NaN  
618                                      NaN  
651                                      NaN  

[622 rows x 4 columns]

### 6.4 Reset index and sort

In [17]:
df_stations = df_stations.reset_index()
df_stations = df_stations.sort_values(by='ID')
df_stations.head()

ID            station_name          x          y  \
0   2     Buckingham Fountain  41.876423 -87.620339   
1   3          Shedd Aquarium  41.867226 -87.615355   
2   4          Burnham Harbor  41.857412 -87.613792   
3   5  State St & Harrison St  41.874053 -87.627716   
4   6          Dusable Harbor  41.886976 -87.612813   

                                  position  
0                  (41.876423, -87.620339)  
1         (41.86722595682, -87.6153553902)  
2  (41.85741178707404, -87.61379152536392)  
3                  (41.874053, -87.627716)  
4                  (41.886976, -87.612813)

## 7 Export

In [18]:
df_stations.to_csv(r'../dataset/chicago_stations.csv', index = False)